In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
activity = pd.read_csv("train_activity.csv").drop("Unnamed: 0", axis=1)
label = pd.read_csv("train_label.csv").drop("Unnamed: 0", axis=1)

In [ ]:
activity.head()

In [ ]:
label.head()

<br></br><br></br><br></br>

In [ ]:
id_lst = sorted(list(label["acc_id"]))

print(len(id_lst))

In [ ]:
min_lst = [0]+ list(activity.min())[3:]

print(len(activity.columns))
print(len(min_lst))
print(min_lst)

In [ ]:
activity_dic = {}

for i in id_lst :
    for idx in range(1,9) : 
        activity_dic[(idx, (int(i)))] = min_lst
        
print(len(activity_dic))

In [ ]:
for data in activity.values :
    activity_dic[(int(data[0]), int(data[1]))] = data[2:]

In [ ]:
activity_lst = [list(y) for y in activity_dic.values()]
activity_lst[0]

In [ ]:
cols = list(activity.columns[2:])
col_dic = {}

for idx, col in enumerate(cols) :
    col_dic[idx] = col 

In [ ]:
activity.sort_values(by=["acc_id", "wk"]).head(20)

In [ ]:
activity2 = pd.DataFrame(activity_lst).rename(columns=col_dic)

print(len(activity2))
activity2.head(20)

In [ ]:
label_dic = {"week":0 , "month" :1, "2month":2, "retained":3}

In [ ]:
label2 = label.sort_values(by="acc_id")
label2["label"] = label2["label"].map(lambda x : label_dic[x])

label2.head()

<br></br><br></br><br></br>

In [ ]:
def one_hot(lst, num_class=4) :
    return np.eye(num_class)[lst]

In [ ]:
total_lst = []
wk = 8

for now in range(0,800001,8) :
    if now == 0 :
        last = now
        continue
    
    total_lst.append(activity_lst[last:now])
    last=now
    
print(len(total_lst))

In [ ]:
total_label = one_hot(label2.label.values)
print(len(total_label))

In [ ]:
training_lst = np.array(total_lst[:80000])
valid_lst = np.array(total_lst[80000:90000])
test_lst = np.array(total_lst[90000:])

training_label = np.array(total_label[:80000])
valid_label = np.array(total_label[80000:90000])
test_label = np.array(total_label[90000:])

In [ ]:
print(len(training_lst[0]))
training_lst[0]

<br></br><br></br><br></br>

# Model

In [ ]:
input_length = 8
input_size = 36
input_class = 4
hidden_layer1 = 64
hidden_layer2 = 128

In [ ]:
class Model1() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc
                 

In [ ]:
class Model2() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model3() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.5)
            cell3 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, dropout2, cell3])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model4() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.5)
            cell3 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, dropout2, cell3])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model5() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense1= tf.layers.dense(inputs=output, units=32)
            dense2 = tf.layers.dense(inputs=dense1, units=input_class)
            self.logits = dense2

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model6() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense1= tf.layers.dense(inputs=output, units=64)
            dense2 = tf.layers.dense(inputs=dense1, units=input_class)
            self.logits = dense2

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model7() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.5)
            cell3 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, dropout2, cell3])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense1= tf.layers.dense(inputs=output, units=32)
            dense2 = tf.layers.dense(inputs=dense1, units=input_class)
            self.logits = dense2

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
class Model8() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.5)
            cell3 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, dropout2, cell3])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense1= tf.layers.dense(inputs=output, units=64)
            dense2 = tf.layers.dense(inputs=dense1, units=input_class)
            self.logits = dense2

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

<br></br><br></br><br></br>

In [ ]:
learning_rate1 = 0.01
learning_rate2 = 0.005
learning_rate3 = 0.002
learning_rate4 = 0.001

total_epoch = 160
batch_size = 500

In [ ]:
train_losses1 = []
train_accs1 = []
valid_losses1 = []
valid_accs1 = []

train_losses2 = []
train_accs2 = []
valid_losses2 = []
valid_accs2 = []

train_losses3 = []
train_accs3 = []
valid_losses3 = []
valid_accs3 = []

train_losses4 = []
train_accs4 = []
valid_losses4 = []
valid_accs4 = []

train_losses5 = []
train_accs5 = []
valid_losses5 = []
valid_accs5 = []

train_losses6 = []
train_accs6 = []
valid_losses6 = []
valid_accs6 = []

train_losses7 = []
train_accs7 = []
valid_losses7 = []
valid_accs7 = []

train_losses8 = []
train_accs8 = []
valid_losses8 = []
valid_accs8 = []

In [ ]:
sess = tf.Session()

model1 = Model1(sess, "model1")
model1.build()

model2 = Model2(sess, "model2")
model2.build()

model3 = Model3(sess, "model3")
model3.build()

model4 = Model4(sess, "model4")
model4.build()

model5 = Model5(sess, "model5")
model5.build()

model6 = Model6(sess, "model6")
model6.build()

model7 = Model7(sess, "model7")
model7.build()

model8 = Model8(sess, "model8")
model8.build()

sess.run(tf.global_variables_initializer())

In [ ]:
print('Learning Started!')
print("")

# train my model
for epoch in range(total_epoch):
    avg_cost1 = 0
    avg_cost2 = 0
    avg_cost3 = 0
    avg_cost4 = 0
    avg_cost5 = 0
    avg_cost6 = 0
    avg_cost7 = 0
    avg_cost8 = 0
    
    total_batch = int(len(training_lst) / batch_size)
    idx = 0
    
    if epoch == 0 :
        learning_rate = learning_rate1
    elif epoch == 40 :
        learning_rate = learning_rate2
    elif epoch == 80 :
        learning_rate = learning_rate3
    elif epoch == 120 :
        learning_rate = learning_rate4

    for i in range(total_batch):
        batch_xs, batch_ys = training_lst[idx:idx+batch_size],training_label[idx:idx+batch_size]
        
        c1, _ = model1.train(batch_xs, batch_ys, learning_rate)
        c2, _ = model2.train(batch_xs, batch_ys, learning_rate)
        c3, _ = model3.train(batch_xs, batch_ys, learning_rate)
        c4, _ = model4.train(batch_xs, batch_ys, learning_rate)
        c5, _ = model5.train(batch_xs, batch_ys, learning_rate)
        c6, _ = model6.train(batch_xs, batch_ys, learning_rate)
        c7, _ = model7.train(batch_xs, batch_ys, learning_rate)
        c8, _ = model8.train(batch_xs, batch_ys, learning_rate)
        
        avg_cost1 += c1 / total_batch
        avg_cost2 += c2 / total_batch
        avg_cost3 += c3 / total_batch
        avg_cost4 += c4 / total_batch
        avg_cost5 += c5 / total_batch
        avg_cost6 += c6 / total_batch
        avg_cost7 += c7 / total_batch
        avg_cost8 += c8 / total_batch
        
        idx += batch_size
        
        if i%55 == 0 :
            print("log1 : ", avg_cost1)
            print("log2 : ", avg_cost2)
            print("log3 : ", avg_cost3)
            print("log4 : ", avg_cost4)
            print("log5 : ", avg_cost5)
            print("log6 : ", avg_cost6)
            print("log7 : ", avg_cost7)
            print("log8 : ", avg_cost8)
            
    #train cost & acc
    cost1, acc1 = model1.evaluate(training_lst, training_label, batch_size = batch_size)
    cost2, acc2 = model2.evaluate(training_lst, training_label, batch_size = batch_size)
    cost3, acc3 = model3.evaluate(training_lst, training_label, batch_size = batch_size)
    cost4, acc4 = model4.evaluate(training_lst, training_label, batch_size = batch_size)
    cost5, acc5 = model5.evaluate(training_lst, training_label, batch_size = batch_size)
    cost6, acc6 = model6.evaluate(training_lst, training_label, batch_size = batch_size)
    cost7, acc7 = model7.evaluate(training_lst, training_label, batch_size = batch_size)
    cost8, acc8 = model8.evaluate(training_lst, training_label, batch_size = batch_size)
    
    train_losses1.append(cost1)
    train_accs1.append(acc1)
    train_losses2.append(cost2)
    train_accs2.append(acc2)
    train_losses3.append(cost3)
    train_accs3.append(acc3)
    train_losses4.append(cost4)
    train_accs4.append(acc4)
    train_losses5.append(cost5)
    train_accs5.append(acc5)
    train_losses6.append(cost6)
    train_accs6.append(acc6)
    train_losses7.append(cost7)
    train_accs7.append(acc7)
    train_losses8.append(cost8)
    train_accs8.append(acc8)
    
    #test cost & acc
    v_cost1, v_acc1 = model1.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost2, v_acc2 = model2.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost3, v_acc3 = model3.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost4, v_acc4 = model4.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost5, v_acc5 = model5.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost6, v_acc6 = model6.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost7, v_acc7 = model7.evaluate(valid_lst, valid_label, batch_size = batch_size)
    v_cost8, v_acc8 = model8.evaluate(valid_lst, valid_label, batch_size = batch_size)
    
    valid_losses1.append(v_cost1)
    valid_accs1.append(v_acc1)
    valid_losses2.append(v_cost2)
    valid_accs2.append(v_acc2)
    valid_losses3.append(v_cost3)
    valid_accs3.append(v_acc3)
    valid_losses4.append(v_cost4)
    valid_accs4.append(v_acc4)
    valid_losses5.append(v_cost5)
    valid_accs5.append(v_acc5)
    valid_losses6.append(v_cost6)
    valid_accs6.append(v_acc6)
    valid_losses7.append(v_cost7)
    valid_accs7.append(v_acc7)
    valid_losses8.append(v_cost8)
    valid_accs8.append(v_acc8)
    
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost1, acc1*100, v_cost1, v_acc1*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost2, acc2*100, v_cost2, v_acc2*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost3, acc3*100, v_cost3, v_acc3*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost4, acc4*100, v_cost4, v_acc4*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost5, acc5*100, v_cost5, v_acc5*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost6, acc6*100, v_cost6, v_acc6*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost7, acc7*100, v_cost7, v_acc7*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost8, acc8*100, v_cost8, v_acc8*100))
    print('Accuracy:', model1.get_accuracy(test_lst, test_label))
    print('Accuracy:', model2.get_accuracy(test_lst, test_label))
    print('Accuracy:', model3.get_accuracy(test_lst, test_label))
    print('Accuracy:', model4.get_accuracy(test_lst, test_label))
    print('Accuracy:', model5.get_accuracy(test_lst, test_label))
    print('Accuracy:', model6.get_accuracy(test_lst, test_label))
    print('Accuracy:', model7.get_accuracy(test_lst, test_label))
    print('Accuracy:', model8.get_accuracy(test_lst, test_label))
    print(" ")

print("")
print('Learning Finished!')

In [ ]:
for idx in range(1,9) :
    plt.plot(eval("train_losses"+str(idx)), label='training'+str(idx))
    plt.plot(eval("valid_losses"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,9) :
    plt.plot(eval("train_accs"+str(idx)), label='training'+str(idx))
    plt.plot(eval("valid_accs"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,9) :
    plt.plot(eval("train_losses"+str(idx)), label='training'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,9) :
    plt.plot(eval("valid_accs"+str(idx)), label='valid'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
tf.reset_default_graph() 

<br></br><br></br><br></br>

- payment 
    - week별 충전 금액
    
- trade
    - week별 교환 신청 횟수
    - week별 교환 받은 횟수
    - 교환 물품 횟수
    
- guild
    - 있었던 길드의 개수
    - 길드원의 평균 activity

- party 
    - week별 party 횟수
    - 파티원의 평균 activity

- 의문점
    - guild가 여러 개인 경우에는 어떻게 처리할건지
    - party와 guild의 정보를 어떻게 할건지
        - 현재 multi input model 알아보는 중, 지금까지 찾은건 맘에 드는 것이 없음
        - party id별, guild id별 정보를 만드는 것이 중요해보임
    - 플레이를 했던 시간대도 굉장히 중요할거같은데 이것을 어떻게 처리해야할지 애매

- 모델링 계획
    - zero padding으로 해보기(현재는 각 column의 최소값)
        - 3주차에 안하는 것과 8주차에 안하는 것은 차이나기 때문에 데이터를 몰아서 넣는 것이 아니라 중간중간에 빈 곳을 padding해줘야 함
    - guild/party등의 정보를 초기 state혹은 input에 concatenate하는 방식
    - CNN 사용
    - AutoEncoder 사용
        - guild/party 등의 정보를 추출할 때 사용
    - 모든 활동 내역을 시간 순으로 나열하여 각 주별로 RNN으로 처리하여 encoding

- 해결책
    - 각 주차에 시작한 사람들끼리 모델을 만들기
        - 1주차에 시작한 사람이랑 5주차에 시작한 사람에 차이를 줘야하기 때문
    - Attention
        - Transformer
    - token
        - 총 4개의 token을 사용(start, end, empty, zero)
        - 비어있는 것을 하나로 합치고, 대신 얼마나 비어있는지를 값으로 지정